# Journal Summarization Notebook for Colab

Written by Kate Habich, pulled from code by myself, Chanteria Milner, and the HuggingFace team.

### Environment Setup

In [1]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install numpy
!pip install tensorflow
!pip install rouge_score
!apt install git-lfs
!git config --global user.email "kate.habich@gmail.com"
!git config --global user.name "Kate Habich"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9e4ababc7b225bda012a076a97195035fbbb6b91a9082071ea3e0a9901832fdb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## Util Functions

In [2]:
import ast
import pandas as pd

def load_file_to_df(
    filepath: str, load_tokenized=False, tokenized_cols=None
) -> pd.DataFrame:
    """
    Loads a file into a dataframe.

    parameters:
        filepath (str): path to the file to load.
        load_tokenized (bool): whether or not to load tokenized data.
        tokenized_cols (list): list of columns to load tokenized data for.

    returns:
        df (pd.DataFrame): dataframe of the file.
    """
    if tokenized_cols is None:
        tokenized_cols = []
    ext = filepath.split(".")[-1].lower()

    if ext in ["pickle", "pkl"]:
        df = pd.read_pickle(filepath)
    elif ext in ["csv", "txt"]:
        df = pd.read_csv(filepath)
    elif ext in ["xlsx", "xls"]:
        df = pd.read_excel(filepath)
    elif ext in ["fea", "feather"]:
        df = pd.read_feather(filepath)
    else:
        raise ValueError(f"File type {ext} not supported.")

    if load_tokenized:
        for col in tokenized_cols:
            df[col] = df[col].apply(ast.literal_eval)

    return df

## Pre-Processing Data

In [6]:
from datasets import Dataset
import pandas as pd
from google.colab import drive

def preprocess_journal_data():
    # Read in data
    # data = load_file_to_df('project/data/proquest_data_cleaned.fea')
    drive.mount('/content/drive')
    data = load_file_to_df('/content/drive/My Drive/UChicago/Machine_Learning/Advanced_ML_Project/Proquest Data/proquest_data_cleaned.fea')

    # Select relevant columns and rows
    limited_data = data.loc[(data['title'].notnull()) &
                            (data['abstract'].notnull()) &
                            (data['doi'].notnull()),
                            ['abstract', 'title', 'doi']].rename(
        columns = {'abstract': 'document',
                   'title':'summary',
                   'doi': 'id'})

    # Cast df to Dataset type for HF model
    preprocessed_data = Dataset.from_pandas(limited_data).remove_columns('__index_level_0__')

    return preprocessed_data

## Fine-Tuning

In [17]:
from datasets import load_dataset, load_metric, DatasetDict
import transformers
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, create_optimizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from functools import partial
import evaluate
import numpy as np
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers.utils import send_example_telemetry
from huggingface_hub import notebook_login
from transformers import T5ForConditionalGeneration
import nltk
nltk.download('punkt')

# notebook_login()
send_example_telemetry("summarization_notebook", framework="tensorflow")

class AbstractSummarizer:
  def __init__(self, training_data, model_save_path, read_from_external = True):
      self.data = training_data
      self.read_from_external = read_from_external
      self.model = self._fine_tune(self.data)
      self.model_save_path = model_save_path

  def _fine_tune(self, data):
      '''
      Create train-test split of data and tokenize.
      '''
      if self.read_from_external:
          model = T5ForConditionalGeneration.from_pretrained(self.model_save_path)
      else:
          train_testvalid = self.data.train_test_split(test_size=0.1)
          test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
          raw_datasets = DatasetDict({
              'train': train_testvalid['train'],
              'test': test_valid['test'],
              'validation': test_valid['train']})

          metric = load_metric("rouge")

      #     # Tokenize text
      #     # TODO: change checkpoint to 'large' or 'base' version eventually
          # model_checkpoint = "google-t5/t5-small"
          model_checkpoint = "t5-small"
          tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

          prefix = "summarize: "
          tokenized_datasets = raw_datasets.map(partial(self._preprocess_function,
                                                        prefix = prefix,
                                                      tokenizer = tokenizer),
                                                batched=True)

      #     # push to hub (comment out if you want)
      #     # model_name = checkpoint.split("/")[-1]
      #     # push_to_hub_model_id = f"{model_name}-finetuned-climate-journals"

          model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
          batch_size = 8
          learning_rate = 2e-5
          weight_decay = 0.01
          num_train_epochs = 1
          data_collator = DataCollatorForSeq2Seq(tokenizer,
                                                model,
                                                return_tensors="np")
          generation_data_collator = DataCollatorForSeq2Seq(tokenizer,
                                                          model=model,
                                                          return_tensors="np",
                                                          pad_to_multiple_of=128)

          tf_train_set = model.prepare_tf_dataset(
                    tokenized_datasets["train"],
                    shuffle=True,
                    batch_size=batch_size,
                    collate_fn=data_collator,
                    )

          tf_valid_set = model.prepare_tf_dataset(
                    tokenized_datasets["validation"],
                    shuffle=False,
                    batch_size=batch_size,
                    collate_fn=data_collator,
                    )
          generation_dataset = model.prepare_tf_dataset(
                    # subset["validation"],
                    tokenized_datasets["validation"],
                    batch_size=8,
                    shuffle=False,
                    collate_fn=generation_data_collator
                )
          # tf_test_set = model.prepare_tf_dataset(
          #           tokenized_datasets["test"],
          #           shuffle=False,
          #           batch_size=batch_size,
          #           collate_fn=data_collator,
          #           )
          optimizer = AdamWeightDecay(learning_rate,
                                      weight_decay)

          model.compile(optimizer=optimizer)

          metric_callback = KerasMetricCallback(partial(self._metric_fn,
                                                        tokenizer = tokenizer,
                                                        metric = metric),
                                                eval_dataset=generation_dataset,
                                                predict_with_generate=True,
                                                use_xla_generation=True)

          model.fit(x=tf_train_set,
                    validation_data=tf_valid_set,
                    epochs=1,
                    callbacks=[metric_callback])

          model.save_pretrained(model_save_path)

      return model


  def _preprocess_function(self, examples, prefix, tokenizer):
      '''
      Prefixes
      '''
      max_input_length = 1920
      max_target_length = 128
      inputs = [prefix + doc for doc in examples["document"]]
      model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

      # Setup the tokenizer for targets
      with tokenizer.as_target_tokenizer():
          labels = tokenizer(
              examples["summary"], max_length=max_target_length, truncation=True
          )

      model_inputs["labels"] = labels["input_ids"]
      return model_inputs


  def _metric_fn(self, eval_predictions, tokenizer, metric):
      predictions, labels = eval_predictions
      decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
      for label in labels:
          label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
      decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
      # Rouge expects a newline after each sentence
      decoded_predictions = [
          "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
      ]
      decoded_labels = [
          "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
      ]
      result = metric.compute(
          predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
      )
      # Extract a few results
      result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
      # Add mean generated length
      prediction_lens = [
          np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
      ]
      result["gen_len"] = np.mean(prediction_lens)

      return result


  def summarize(self, document):
      '''
      Summarizes string using model output
      '''
      model_checkpoint = "t5-small"
      tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
      prompt = "summarize: " + document
      tokenized = tokenizer([prompt], return_tensors='np')
      out = self.model.generate(**tokenized, max_length=128)

      with tokenizer.as_target_tokenizer():
          summary = tokenizer.decode(out[0])

      return summary


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
subset = preprocess_journal_data().train_test_split(train_size=20)['train']
full_data = preprocess_journal_data()
model_save_path = '/content/drive/My Drive/UChicago/Machine_Learning/models/abstract_model/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
summarizer = AbstractSummarizer(subset,
                                model_save_path,
                                False)

# TODO: Import model from saved here
document = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled.'
summary = summarizer.summarize(document)

print(summary)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


2/2 [==============================] - ETA: 0s - loss: 3.2169  

/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  return py_builtins.overload_of(f)(*args)


2/2 [==============================] - 190s 75s/step - loss: 3.2169 - val_loss: 5.7648 - rouge1: 14.2857 - rouge2: 0.0000e+00 - rougeL: 14.2857 - rougeLsum: 14.2857 - gen_len: 19.0000
<pad> repairs are ongoing in the area and many roads in peeblesshire remain badly affected. the water breached a retaining wall, flooding many commercial properties. the water was flooded by flooding in the town of Newton Stewart.</s>


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import T5ForConditionalGeneration
saved_model = T5ForConditionalGeneration.from_pretrained(model_save_path)
